# 第3章 k近邻法
K近邻算法本身是个很简单的算法，某些时候甚至不能称之为机器学习算法，因为它没有“学习”的过程，仅仅通过判定和要预测的$x$相近的点都呈现的表征来确定$x$的对应类标$y$就可以了，但是《统计学习方法》这本书使用了KD-tree的方法来寻找最近邻，其主要目的是在数据量比较大的时候能够优化查找效率，从原理来讲，树结构的查找的时间复杂度$O\left( logN \right)$比线性查找的复杂度$O\left( N \right)$低，因此效率比较高。
## 3.1k近邻算法(KNN)
>输入:训练数据集
    $$ T = \{ (x_1,y_1),(x_2,y_2),...,(x_N,y_N) \}$$
    其中, $x_i \in \mathcal{X} \subseteq R^n$为实例的特征向量,$y_i \in \mathcal{Y} =\{ c_1,c_2,...,c_K\}$为实例的类别, $i = 1,2,...N$; 实例特征向量$x$.  
输出:实例$x$所属类别$y$
* 根据距离度量,在训练集$T$中找到与$x$最近邻的$k$个点,涵盖这$k$个点的邻域记作$N_k(x)$
* 在$N_k(x)$中根据分类决策规则(如多数表决)决定$x$的类别$y$
$$ y = arg \mathop{\min}_{c_j} \sum_{x_i \in N_k(x)} I(y_i = c_j),i=1,2,...,N;j=1,2,...,k$$
上式中,$I$为指示函数,当$y_i = c_j$的时候,$I$为1,否则置0
(注:关于下标的另外一种写法:$\arg\underset{h\in\mathcal{X}}{\min}$)

## 3.2距离度量方法以及定义
>* $L_p$ distance: $L_p$距离
$$L_p(x_i,x_j) = (\sum^n_{l=1} {\left| x^{(l)}_i - x^{(l)}_j \right|}^p)^\frac{1}{p}$$
* Euclidean distance: 欧式距离
$$L_2(x_i,x_j) = (\sum^n_{l=1} {\left| x^{(l)}_i - x^{(l)}_j \right|}^2)^\frac{1}{2}$$
* Manhattan distance: 曼哈顿距离
$$L_1(x_i,x_j) = \sum^n_{l=1} {\left| x^{(l)}_i - x^{(l)}_j \right|}$$

In [1]:
def calDis(x_1,x_2,dis_type = None):
    """
    距离度量公式,包含2种,分别记为'E','M',E为欧式距离,M是曼哈顿距离
    x_1,x_2为两个向量S
    """
    distance = 0
    if (dis_type == None)|(dis_type == 'E'):
        distance = math.sqrt(sum((x_1-x_2)**2))
    elif (dis_type == 'M'):
        distance = sum(abs(x_1-x_2))
    else:
        print('WARNING: 函数calDis(self,x_1,x_2,dis_type)没有dis_type='+ str(dis_type) +' 这种距离度量方法,请重新定义dis_type=E或者M')
    return distance

In [2]:
# 测试距离计算
import numpy as np
import math
a = np.array([1,2])
b = np.array([2,1])
print('欧式距离为:' ,calDis(a,b,'E'))
print('曼哈顿距离为:' , calDis(a,b,'M'))

欧式距离为: 1.4142135623730951
曼哈顿距离为: 2


## 3.3 k近邻的实现: kd树
k近邻最简单的方法是线性扫描数据,但是这种方法在数据量很大的时候计算非常耗时,这里使用kd树来优化效率.另外,KD树可以理解为是一种对高维空间的数据点进行划分的特殊数据结构，主要应用就是高维空间的数据查找.  
> 方法:  依次选择坐标轴对空间切分(也即对每个维度进行切分),选择在选定坐标轴的中位数作为切分点,此时,kd树是平衡的,但不一定是最优的.  

补充:
* 与距离度量有关的算法,应该对数据进行归一化处理,否则会导致数据的某些特征因为数值较大而被加强,而更有区分度的某些特征可能因为数据过小导致不被重视.
* 这里kdtree中的k和knn的k含义不一样

### 3.3.1 根据数据 $X$ 定义KDTree 
关于定义KD树的思路和讲解,可以参考[这里](https://blog.csdn.net/yan456jie/article/details/52074141)
>定义树节点kdnode，注：由于发现在找到最终节点个向上递归的时候比较难以实现，这里定义树节点为带有父节点的node，另外，由于kdtree算法中需要反复到另外一个子树上找到是否存在最近节点，所以这里再为节点添加一个属性“树分支标记flag，用于标记该子树是父节点的哪个子树，这里flag的取值为‘L’或者‘R’，分别为左子树和右子树”。

In [16]:
class kdnode:
    def __init__(self,parent = None, key = None, left = None, right = None,flag = None):
        self.parent = parent # 父节点
        self.key = key # 值
        self.left = left #  子树
        self.right = right # 子树
        self.flag = flag # 标记是上一层的左右子树（l：左，r右）

>递归构建树

In [17]:
import matplotlib.pyplot as plt
def createkdtree(X,parent = None,startcolumn = None,display = None,flag = None):
    """
    X: 数据本身,为矩阵形式
    parent: 父节点
    startcolumn: 开始计算的属性列序号,所以startcolumn的值应该小于属性数
    display: 显示构建过程, 0:不显示,1:显示. 默认为0
    flag: 子树位置标记，用于标记目前的树是父节点的哪个子树（L或者R）
    以上方法在初次调用的时候，只需要输入所有样本矩阵X
    
    """
#     plt.figure
    if startcolumn == None:
        startcolumn = 0
        tree = kdnode
        tree.parent = None
        tree.flag = None
#         if X.shape[1] == 2:
#             rangexmin = np.min(X[:,0])
#             rangexmax = np.max(X[:,0])
#             rangeymin = np.min(X[:,1])
#             rangeymax = np.max(X[:,1])
#             plt.xlim((rangexmin, rangexmax))
#             plt.ylim((rangeymin, rangeymax))
    tree = kdnode() # 定义当前节点为新的树节点
    X_length = len(X) # 查看当前X还有多少数据
    if X_length == 0: # 如果X集合里面已经不存在样本了,则不再构建树了,直接跳出
        return
    sorted_X = X[X.argsort(0)[:,startcolumn]] # 以startcolumn列属性为基准,对X进行排序
    m,n = sorted_X.shape # 找到X的行数和列数
    midindex = int(np.floor(m/2)) # 找到中位数(实际可以理解为中位向量)
    tree.parent = parent
    tree.flag = flag
    tree.key = sorted_X[midindex,:] # 将当前的中位数赋值给节点
    sub_left = sorted_X[:midindex,:] # 定义在这之前的为左孩子集合
    sub_right = sorted_X[(midindex+1):,:] # 定义在这之后的为右孩子集合
    if display == True:    # 决定是否输出显示孩子关系,1,则显示
        if tree.parent != None:
            print('>>>当前节点值为:\n' , tree.key,'\n*左孩子集合:\n',(sub_left),'\n*右孩子集合:\n',(sub_right),'\n*父节点是：',(tree.parent.key))
        if tree.parent == None:
            print('>>>当前节点值为:\n' , tree.key,'\n*左孩子集合:\n',(sub_left),'\n*右孩子集合:\n',(sub_right))
    if len(sub_left)>0:    # 如果左孩子集合不为空,在下一轮中对左孩子集合以第startcolumn+1列属性为基准进行递归对建立子树.
        tree.left = createkdtree(sub_left,tree,np.mod(startcolumn+1,n),display,'l') 
    if len(sub_right)>0:   # 如果右孩子集合不为空,在下一轮中对右孩子集合以第startcolumn+1列属性为基准进行递归对建立子树.
        tree.right = createkdtree(sub_right,tree,np.mod(startcolumn+1,n),display,'r')
#     plt.show()
    return tree # 返回树

>测试教材用例

In [18]:
data = np.array([[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]])# ,[4,5],[3,6]
tree = createkdtree(data)

### 3.3.2 用kdtree实现最近邻搜索
> 输入:已经构造好的kd树,目标点$x$，目标点与kdtree同维度  
输出:$x$的最近邻

>  kdtree树最近邻搜索思想具体步骤：  
* 1.在kd树中递归向下索引，直到索引到了的叶节点  
* 2.以1.中的节点记为“当前最近点”
* 3.递归向上回退，在每个节点执行以下操作：
>>* 如果该节点保存的实例点比“当前最近点”距离目标点更近，则标记该实例点为“当前最近点”  
>>* 当前最近点一定存在于该节点一个子节点对应的区域。检查该子节点的父节点的另外一个子节点对应的区域是否有更近的点，开始以$x$为圆心，以到当前最近点之间的距离为半径，画圆圈，判断与上一层的划分点所确定的超平面是否相交
>>>* 如果相交: 另外一个子节点对应的区域可能有更近的点
>>>* 如果不相交: 向上回退
* 4.回退到根节点

In [45]:
def getnearestNode(kdtree,x,l = None,mindistance = None,  recursive_traversal = True, backtrace_up = None, backtrace_down = None):
    """
    kdtree: 输入树
    x： 要搜索的点的紧邻
    了： 标记是第几个维度
    recursive_traversal: 向下递归搜索叶子节点
    mindistance: 标记最小值
    backtrace_up： 标记找到最近节点之后向上递归
    backtrace_down： 标记找到最近节点之后向下递归
    
    搜索的算法思想: 首先，顺着节点往下走，根据值不停地选择左右节点子树找到最终的叶子节点;
    """
    
    mindistance = mindistance
    if l == None: # 初始化
        l = 0
        mindistance = -1
        recursive_traversal = True
    else:
        l = l
    if recursive_traversal == True: #向下递归
        distance = calDis(np.array(x),np.array(kdtree.key),'E') #这里调用距离度量方法，这里是欧式距离 
        print(x,'与点',kdtree.key,'的距离为：', distance)
        if kdtree.key[l] > x[l]: # 如果样本的第l个维度的值小于节点的第l个维度的值，说明要去左子树上继续找
            if kdtree.left != None:
                getnearestNode(kdtree.left,x,np.mod(l+1,x.shape[0]),distance,recursive_traversal)  
            elif kdtree.left == None:
                print('最近叶子节点为',kdtree.key)
                mindistance = calDis(np.array(x),np.array(kdtree.key),'E')
                print('=====结束查找叶子节点=====')
                recursive_traversal = False # 结束递归
                backtrace_up = True # 开启回溯
                backtrace_down = False
        if kdtree.key[l] <= x[l]: # 如果样本的第l个维度的值大于或者等于节点的第l个维度的值，说明要去右子树上继续找
            if kdtree.right != None:
                getnearestNode(kdtree.right,x,np.mod(l+1,x.shape[0]),distance,recursive_traversal) 
            elif kdtree.right == None:
                print('最近叶子节点为',kdtree.key)
                mindistance = calDis(np.array(x),np.array(kdtree.key),'E')
                print('=====结束查找叶子节点=====')
                recursive_traversal = False # 结束递归
                backtrace_up = True # 开启回溯
                backtrace_down = False
    # 以上部分，即查找最临近的一个节点，结束上面的部分，接下来开始向上回溯找
    if recursive_traversal == False and backtrace_up == True and backtrace_down == False: #向上递归
        print(recursive_traversal,backtrace_up,backtrace_down)
        if kdtree.parent != None: # 如果父节点不为空
            panelindex = np.mod(l+x.shape[0]-1,x.shape[0])
            print('上一层是第',np.mod(l+x.shape[0]-1,x.shape[0]), 
                  '划分，分割超平面由第',panelindex,'个元素的参数决定')
            dot2panel = np.abs(kdtree.parent.key[panelindex] - x[panelindex])# 计算点到这个分割超平面之间的距离
            r = mindistance # 另半径等于之前的最短
            print('与',kdtree.parent.key,'半径：',r,'， 点面距离：',dot2panel)
            if r > dot2panel: # 半径与上一层划分超平面相交
                if (mindistance>=0): # 并且之前的最小距离是有效值
                    backtrace_up = False
                    backtrace_down = True
                    print('==========\n半径和上一层超平面相交,上一层的另外一个分支可能存在更近的点，需要递归另一棵子树')
                    print('当前子树在上一节点的分支标记为：',kdtree.flag,'，所以要去另外一个分支进行递归查找')
                    if kdtree.flag == 'l':
                        print('go to r!')
                        anothertree = kdtree.parent.right
                        getnearestNode(anothertree,x,np.mod(l+x.shape[0]-1,x.shape[0]), mindistance,
                                       recursive_traversal,backtrace_up,backtrace_down)
                    elif kdtree.flag == 'r':
                        print('go to l!')
                        anothertree = kdtree.parent.left
                        getnearestNode(anothertree,x,np.mod(l+x.shape[0]-1,x.shape[0]), mindistance,
                                       recursive_traversal,backtrace_up,backtrace_down)
            elif r < dot2panel: # 半径与上一层划分超平面不相交
                mindistance = r
                if (mindistance>=0): # 并且之前的最小距离是有效值
                    print('==========\n往父节点找')
                    backtrace_up = True
                    backtrace_down = False
                    getnearestNode(kdtree.parent,x,np.mod(l+x.shape[0]-1,x.shape[0]), mindistance,
                                   recursive_traversal,backtrace_up,backtrace_down)
        elif kdtree.parent == None:
            print('over')
    if recursive_traversal == False and backtrace_down == True and backtrace_up == False:
        print(recursive_traversal,backtrace_up,backtrace_down)
        print('子树')

In [46]:
getnearestNode(tree,np.array([6.1,2.1]))

[ 6.1  2.1] 与点 [7 2] 的距离为： 0.905538513813742
[ 6.1  2.1] 与点 [5 4] 的距离为： 2.195449840010015
[ 6.1  2.1] 与点 [2 3] 的距离为： 4.197618372363071
最近叶子节点为 [2 3]
=====结束查找叶子节点=====
False True False
上一层是第 1 划分，分割超平面由第 1 个元素的参数决定
与 [5 4] 半径： 4.197618372363071 ， 点面距离： 1.9
半径和上一层超平面相交,上一层的另外一个分支可能存在更近的点，需要递归另一棵子树
当前子树在上一节点的分支标记为： l ，所以要去另外一个分支进行递归查找
go to r!
